In [35]:
import streamlit as st

print("hello")
# Import necessary libraries
import ee
import geopy

# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project="1030170391022")

# Function to get the coordinates of a city using Geopy
def get_city_coordinates(city_name):
    geolocator = geopy.Photon(user_agent="pacc-24")
    location = geolocator.geocode(city_name)
    return location.latitude, location.longitude


# Function to get climate data for a given city
def get_climate_data(city_name):
    # Get city coordinates
    lat, lon = get_city_coordinates(city_name)

    # Define the point of interest
    point = ee.Geometry.Point(lon, lat)

    # Load a dataset, for example, the ERA5 Daily Aggregate dataset for temperature
    dataset = ee.ImageCollection('ECMWF/ERA5/DAILY') \
        .filterBounds(point) \
        .filterDate('2023-01-01', '2023-12-31')

    # Select the temperature band (2m air temperature)
    temperature = dataset.select('mean_2m_air_temperature')

    # Calculate the mean temperature for the year
    mean_temp = temperature.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=1000
    )
    print(f"-*100\n{lat, lon}")
    print(f"-*100\n{dataset}")
    print(f"-*100\n{temperature}")
    print(f"-*100\n{mean_temp}")
    # Get the temperature in Celsius
    # temp_celsius = mean_temp.get('mean_2m_air_temperature').getInfo() - 273.15

    return mean_temp

def get_future_temperature(city_name):
    # Get the coordinates of the city using geopy.
    # latitude, longitude = get_city_coordinates(city_name)

    # Define the point of interest.
    # point = ee.Geometry.Point([longitude, latitude])

    # Load the GFS forecast dataset.
    dataset = ee.ImageCollection('NOAA/GFS0P25')

    # Filter the dataset to a specific forecast time.
    # Set the forecast time to today at 12 noon.
    forecast_time = ee.Date('2024-07-01T09:00:00')
    forecast_image = dataset.filter(ee.Filter.date(forecast_time)).first()

    # Select the temperature band.
    temperature_band = forecast_image.select('temperature_2m_above_ground')
    return temperature_band


def get_from_band(temperature_band, point):
        # Get the temperature value at the specified point.
    temperature = temperature_band.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=1000
    ).get('temperature_2m_above_ground').getInfo()

    return temperature


hello


In [36]:
# Example usage
city_name = 'New Delhi'

In [37]:
latitude, longitude = get_city_coordinates(city_name)
point = ee.Geometry.Point([latitude, longitude])

In [38]:
temperature_band = get_future_temperature(city_name)

In [39]:
temperature = get_from_band(temperature_band, point)
print(temperature)

EEException: Image.select: Parameter 'input' is required.

In [ ]:

# Assuming 'mean_temp' is the Earth Engine Dictionary object
mean_temp_dict = ee.Dictionary(temperature)

# Extract the temperature in Kelvin using the .get() method
temp_kelvin = mean_temp_dict.get('mean_2m_air_temperature').getInfo()

# Convert the temperature from Kelvin to Celsius
if temp_kelvin is not None:
    temp_celsius = temp_kelvin - 273.15
    print(f'The mean temperature is {temp_celsius:.2f}°C')
else:
    print('Temperature data not available')